In [ ]:
# -----------------------------------------------------------------STEPS------------------------------------------------------------------
# convert to frames with mog
# write frames to disk
# make ds_train,ds_test,ds_val using tf dataset
# make model
# compile model
# fit model
# plot stats

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf


In [ ]:
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

In [ ]:
import os

data_dir = '/content/drive/Shareddrives/Final Year Project/Datasets/Mini-RWF-2000'
!ls '$data_dir'
output_dir='/content/drive/Shareddrives/Final Year Project/Datasets/Processed'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
img_height , img_width = 128, 128
seq_len = 30 # number of frames per video
classes = ["Fight", "NonFight"]

Fight  NonFight


In [ ]:
import os
import cv2
import tqdm as tqdm
import tqdm.notebook as tq
fgbg = cv2.createBackgroundSubtractorMOG2(
    varThreshold=15,
    detectShadows=True
)

def get_mask(frame):
    mog_mask = fgbg.apply(frame)
    median_blur_mask = cv2.medianBlur(mog_mask, 5)
    bilateral_filter_mask = cv2.bilateralFilter(median_blur_mask, 9, 75, 75)
    gaussian_blur_mask = cv2.GaussianBlur(bilateral_filter_mask, (13, 13), 5)

    return gaussian_blur_mask


def frames_extraction(video_path,c,filename):
    frames = []
    cap = cv2.VideoCapture(video_path)

    frame_count = 1
    selected_frames=0
    print(video_path,end="\n\n")

    while True:
        success, frame = cap.read()

        if not success or frame is None:
            break

        if frame_count%5==0:
          selected_frames=selected_frames+1
          new_frame = get_mask(frame)
          save_path=os.path.join(output_dir,c)
          save_path +="/"+ filename.split('.')[0] + '_frame' + str(frame_count) + '.jpg'
          # print(save_path)
          cv2.imwrite(save_path,new_frame)
          # cv2.imshow('Novel Preprocessed Frame', new_frame)
          frames.append(new_frame)

        k = cv2.waitKey(30)
        if selected_frames >= seq_len or k == 27 or k == ord('q'):
            break

        frame_count += 1

    cap.release()
    # cv2.destroyAllWindows()

    return frames

def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        # print(c)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir+"")
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in tq.tqdm(files_list):
            frames = frames_extraction(os.path.join(input_dir, c, f),c,f)
           
            if len(frames) == seq_len:
                X.append(frames)
                y = [0] * len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    # X = np.asarray(X)
    # Y = np.asarray(Y)

    # X = np.expand_dims(X, -1);
    # Y = np.expand_dims(Y, -1);


    return X, Y

In [ ]:
with tf.device('/device:GPU:0'):
  x,y=create_data(data_dir)

In [ ]:
ds_train=tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/Shareddrives/Final Year Project/Datasets/Processed',
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=seq_len,
    image_size=(img_height,img_width),
    subset='training',
    validation_split=0.2,
    seed=123
)
ds_val=tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/Shareddrives/Final Year Project/Datasets/Processed',
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=seq_len,
    image_size=(img_height,img_width),
    subset='validation',
    validation_split=0.2,
    seed=123
)
print("\nClass names: {0}".format(ds_train.class_names))

Found 360 files belonging to 2 classes.
Using 288 files for training.
Found 360 files belonging to 2 classes.
Using 72 files for validation.

Class names: ['Fight', 'NonFight']


In [ ]:
for image_batch, labels_batch in ds_train:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(30, 128, 128, 3)
(30,)


In [ ]:
from tensorflow.keras import layers
import numpy as np

normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = ds_train.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.9893536


In [ ]:
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models

In [ ]:
# --------------------------------------------copied ver 1--------------------------------------------#
# model = Sequential()
# model.add(TimeDistributed(Conv2D(64, 5, activation='relu', padding='same', name='conv1', input_shape=??))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='pool1')))

# model.add(TimeDistributed(Conv2D(64, 5, activation='relu', padding='same', name='conv2'))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='pool2')))

# model.add(TimeDistributed(Conv2D(64, 5, activation='relu', padding='same', name='conv3'))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='pool3')))

# model.add(TimeDistributed(Conv2D(64, 5, activation='relu', padding='same', name='conv4'))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='pool4')))


# model.add(TimeDistributed(Flatten()))
# model.add(LSTM(256, return_sequences=False, dropout=0.5))
# model.add(Dense(1, activation='sigmoid'))

# --------------------------------------------copied ver 2--------------------------------------------#

model = models.Sequential()
model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),)
model.add(Conv2D(128, (3, 3), strides=(1,1),activation=tf.nn.relu,))
model.add(Conv2D(64, (3, 3), strides=(1,1),activation=tf.nn.relu))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3, 3), strides=(1,1),activation=tf.nn.relu))
model.add(Conv2D(32, (3, 3), strides=(1,1),activation=tf.nn.relu))
# model.add(MaxPooling2D(2,2))
# model.add(BatchNormalization())


model.add(Flatten())
# model.add(Dropout(0.2))

# model.add(LSTM(32,return_sequences=False,dropout=0.2)) # used 32 units   # got an error here
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(64,activation=tf.nn.relu))

model.add(Dropout(0.2))
model.add(Dense(32,activation=tf.nn.relu))
model.add(Dense(1, activation=tf.nn.sigmoid))


model.summary()

In [ ]:
model = models.Sequential()

model.add(
    TimeDistributed(
        Conv2D(64, (3, 3), activation=tf.nn.relu), 
        input_shape=(seq_len, img_height,img_width, 3)
    )
)
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))

model.add(TimeDistributed(Conv2D(128, (4,4), activation=tf.nn.relu)))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (4,4), activation=tf.nn.relu)))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

# extract features and dropout 
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))

# input to LSTM
model.add(LSTM(256, return_sequences=False, dropout=0.5))

# classifier with sigmoid activation for multilabel
model.add(Dense(2, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_21 (TimeDis (None, 30, 126, 126, 64)  1792      
_________________________________________________________________
time_distributed_22 (TimeDis (None, 30, 125, 125, 64)  0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, 30, 122, 122, 128) 131200    
_________________________________________________________________
time_distributed_24 (TimeDis (None, 30, 61, 61, 128)   0         
_________________________________________________________________
time_distributed_25 (TimeDis (None, 30, 58, 58, 256)   524544    
_________________________________________________________________
time_distributed_26 (TimeDis (None, 30, 29, 29, 256)   0         
_________________________________________________________________
time_distributed_27 (TimeDis (None, 30, 215296)       

In [ ]:
import keras as keras
from keras import optimizers
import tensorflow_addons as tfa

tqdm_callback = tfa.callbacks.TQDMProgressBar()
callbacks_list=[
                tqdm_callback,
                keras.callbacks.EarlyStopping(monitor='acc',patience=3),
                keras.callbacks.ModelCheckpoint(
                    filepath='cnn_lstm_model.h5',
                    monitor='val_loss',
                    save_best_only=True),
                keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 3)
               ]


optimizer=optimizers.RMSprop(lr=0.01)
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['acc'])

In [ ]:
# Training:
history=model.fit(ds_train,validation_data=ds_val,batch_size=seq_len,epochs=20,)
                      #  callbacks=callbacks_list)

Epoch 1/20


ValueError: ignored